<a href="https://colab.research.google.com/github/vyshali2419/NLP-2403A52028/blob/main/NLP_LAB_ASS_6_(2403A52028_%2CBATCH_02).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install required libraries (run only once in Colab)
!pip install nltk pandas

import pandas as pd
import numpy as np
import re
import nltk
from collections import defaultdict, Counter

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
# Load dataset
df = pd.read_csv('/content/Twitter_Data.csv')

# Display first 5 rows
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [8]:
# Check column names
df.columns


Index(['clean_text', 'category'], dtype='object')

In [9]:
# Assuming tweet text column name is 'clean_text'
tweets = df['clean_text'].dropna().astype(str)
tweets.head()


,clean_text
0,when modi promised “minimum government maximum...
1,talk all the nonsense and continue all the dra...
2,what did just say vote for modi welcome bjp t...
3,asking his supporters prefix chowkidar their n...
4,answer who among these the most powerful world...


In [10]:
def clean_tweet(text):
    text = re.sub(r"http\S+", "", text)     # Remove URLs
    text = re.sub(r"@\w+", "", text)        # Remove mentions
    text = re.sub(r"#\w+", "", text)        # Remove hashtags
    text = re.sub(r"[^a-zA-Z\s]", "", text) # Remove special chars
    text = text.lower()
    return text.strip()

tweets_clean = tweets.apply(clean_tweet)
tweets_clean.head()


,clean_text
0,when modi promised minimum government maximum ...
1,talk all the nonsense and continue all the dra...
2,what did just say vote for modi welcome bjp t...
3,asking his supporters prefix chowkidar their n...
4,answer who among these the most powerful world...


In [13]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

tagged_sentences = []

for tweet in tweets_clean[:500]:   # Using first 500 tweets for speed
    tokens = nltk.word_tokenize(tweet)
    tags = nltk.pos_tag(tokens)
    tagged_sentences.append(tags)

tagged_sentences[:2]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


[[('when', 'WRB'),
  ('modi', 'NN'),
  ('promised', 'VBD'),
  ('minimum', 'JJ'),
  ('government', 'NN'),
  ('maximum', 'JJ'),
  ('governance', 'NN'),
  ('expected', 'VBD'),
  ('him', 'PRP'),
  ('begin', 'VB'),
  ('the', 'DT'),
  ('difficult', 'JJ'),
  ('job', 'NN'),
  ('reforming', 'VBG'),
  ('the', 'DT'),
  ('state', 'NN'),
  ('why', 'WRB'),
  ('does', 'VBZ'),
  ('take', 'VB'),
  ('years', 'NNS'),
  ('get', 'VB'),
  ('justice', 'NN'),
  ('state', 'NN'),
  ('should', 'MD'),
  ('and', 'CC'),
  ('not', 'RB'),
  ('business', 'NN'),
  ('and', 'CC'),
  ('should', 'MD'),
  ('exit', 'VB'),
  ('psus', 'NN'),
  ('and', 'CC'),
  ('temples', 'NNS')],
 [('talk', 'NN'),
  ('all', 'PDT'),
  ('the', 'DT'),
  ('nonsense', 'NN'),
  ('and', 'CC'),
  ('continue', 'VB'),
  ('all', 'PDT'),
  ('the', 'DT'),
  ('drama', 'NN'),
  ('will', 'MD'),
  ('vote', 'VB'),
  ('for', 'IN'),
  ('modi', 'NN')]]

In [14]:
transition_counts = defaultdict(Counter)

for sent in tagged_sentences:
    tags = ['<s>'] + [tag for word, tag in sent] + ['</s>']
    for i in range(len(tags)-1):
        transition_counts[tags[i]][tags[i+1]] += 1

transition_probs = {}

for prev_tag in transition_counts:
    total = sum(transition_counts[prev_tag].values())
    transition_probs[prev_tag] = {tag: count/total
                                  for tag, count in transition_counts[prev_tag].items()}

transition_probs['<s>']


{'WRB': 0.034,
 'NN': 0.378,
 'WP': 0.008,
 'VBG': 0.02,
 'JJ': 0.114,
 'DT': 0.062,
 'IN': 0.06,
 'NNS': 0.066,
 'CD': 0.016,
 'VBD': 0.01,
 'VBZ': 0.014,
 'RB': 0.084,
 'VB': 0.038,
 'PRP$': 0.01,
 'JJR': 0.002,
 'MD': 0.024,
 'PRP': 0.016,
 'VBN': 0.014,
 'VBP': 0.008,
 'CC': 0.018,
 'EX': 0.002,
 'RBR': 0.002}

In [15]:
emission_counts = defaultdict(Counter)

for sent in tagged_sentences:
    for word, tag in sent:
        emission_counts[tag][word] += 1

emission_probs = {}

for tag in emission_counts:
    total = sum(emission_counts[tag].values())
    emission_probs[tag] = {word: count/total
                           for word, count in emission_counts[tag].items()}

list(emission_probs.keys())[:10]


['WRB', 'NN', 'VBD', 'JJ', 'PRP', 'VB', 'DT', 'VBG', 'VBZ', 'NNS']

In [16]:
print("Transition Probabilities Sample:")
for k in list(transition_probs.keys())[:5]:
    print(k, "->", transition_probs[k])

print("\nEmission Probabilities Sample:")
for k in list(emission_probs.keys())[:5]:
    print(k, "->", list(emission_probs[k].items())[:5])


Transition Probabilities Sample:
<s> -> {'WRB': 0.034, 'NN': 0.378, 'WP': 0.008, 'VBG': 0.02, 'JJ': 0.114, 'DT': 0.062, 'IN': 0.06, 'NNS': 0.066, 'CD': 0.016, 'VBD': 0.01, 'VBZ': 0.014, 'RB': 0.084, 'VB': 0.038, 'PRP$': 0.01, 'JJR': 0.002, 'MD': 0.024, 'PRP': 0.016, 'VBN': 0.014, 'VBP': 0.008, 'CC': 0.018, 'EX': 0.002, 'RBR': 0.002}
WRB -> {'NN': 0.23711340206185566, 'VBZ': 0.010309278350515464, 'JJS': 0.010309278350515464, 'JJ': 0.20618556701030927, 'VBN': 0.05154639175257732, 'VBP': 0.08247422680412371, 'PRP': 0.061855670103092786, 'DT': 0.061855670103092786, 'RB': 0.09278350515463918, 'VB': 0.020618556701030927, 'NNS': 0.05154639175257732, 'MD': 0.061855670103092786, 'PRP$': 0.030927835051546393, 'EX': 0.010309278350515464, '</s>': 0.010309278350515464}
NN -> {'VBD': 0.05525040387722133, 'JJ': 0.04975767366720517, 'VBG': 0.02294022617124394, 'WRB': 0.010339256865912763, 'NN': 0.34022617124394183, 'MD': 0.028109854604200322, 'CC': 0.042972536348949916, 'PDT': 0.0009693053311793214, '

In [17]:
word_freq = Counter()

for sent in tagged_sentences:
    for word, tag in sent:
        word_freq[word] += 1

rare_words = [w for w, f in word_freq.items() if f == 1]

print("Number of Rare Words:", len(rare_words))
print("Sample Rare Words:", rare_words[:20])


Number of Rare Words: 2003
Sample Rare Words: ['maximum', 'difficult', 'reforming', 'temples', 'drama', 'main', 'relax', 'prefix', 'names', 'service', 'confusion', 'crustal', 'crass', 'nonsensical', 'abuses', 'putin', 'tho', 'refresh', 'maarkefir', 'surat']


In [19]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    for s in states:
        V[0][s] = start_p.get(s,1e-6) * emit_p.get(s,{}).get(obs[0],1e-6)
        path[s] = [s]

    for t in range(1, len(obs)):
        V.append({})
        new_path = {}

        for curr in states:
            prob, state = max(
                (V[t-1][prev] * trans_p.get(prev,{}).get(curr,1e-6) * emit_p.get(curr,{}).get(obs[t],1e-6), prev)
                for prev in states
            )
            V[t][curr] = prob
            new_path[curr] = path[state] + [curr]

        path = new_path

    prob, state = max((V[-1][s], s) for s in states)
    return path[state]


In [20]:
test_tweet = "this movie is awesome"
test_tokens = test_tweet.split()

states = list(emission_probs.keys())

start_prob = transition_probs['<s>']

predicted_tags = viterbi(test_tokens, states, start_prob, transition_probs, emission_probs)

print("Tweet:", test_tokens)
print("Predicted POS Tags:", predicted_tags)


Tweet: ['this', 'movie', 'is', 'awesome']
Predicted POS Tags: ['DT', 'NN', 'NN', 'NN']
